# Chapter 1 - Getting Started with ANDES

## Introduction

1. What is ANDES?

    ANDES is an open-source Python library for power system modeling, computation, analysis, and control. It supports power flows calculation, transient stability simulation, and small-signal stability analysis for transmission systems. ANDES implements a symbolic-numeric framework for rapid prototyping of differential-algebraic equation-based models. In this framework, a comprehensive library of models is developed, including the full second-generation renewable models. Models in ANDES have been verified with commercial software.

    ANDES is the core simulation engine for the CURENT Largescale Testbed (LTB). More information about CURENT LTB can be found at the [LTB Repository](https://github.com/CURENT/ltb2).

    [ANDES Documentation](https://docs.andes.app/en/latest/) is the main reference for ANDES.

1. Have question or feedback of ANDES?

    You can post it on the [Discussion](https://github.com/cuihantao/andes/discussions).

1. Need reference of existing models?

    All models in ANDES are documented in the [Model reference](https://docs.andes.app/en/latest/modelref.html).

    For further reference such as transfer function diagrams, most of them can be found online such as **ESIG** and **PowerWorld**.

## Objective

In this chapter, you will learn the basic usage of ANDES:
- Run simulation
- Plot results
- Look up documentation

## Submission

After complete the chapter, please save and submit this jupyter notebook file in **CANVAS** with **FORMATTED** name:

`FirstName_LastName_NetID_ChID.ipynb`, for example, `Tim_Cook_tcook3_Ch1.ipynb`.

# Preparation for a Development Installation

Before you start, you will need to log on https://ltb.curent.utk.edu/. You will
see an Certificate Error. Click "Advanced" near the lower-left corner, and then
"Proceed to ltb.curent.utk.edu". If you haven't done so, please create a GitHub
account and share it with Jinning Wang (jwang175@vols.utk.edu), so that he can
give you access. 

Take a moment to familiarize yourself with the JupyterLab interface. Watch a
tutorial here: https://www.youtube.com/watch?v=A5YyoCKxEOU. 

On JupyterLab, use the blue "+" button to create a new Terminal. Run
the following commands in the new **terminal** (use Ctrl+v to paste the
commands): 

```bash

mkdir -p ~/repos && cd ~/repos

git clone --single-branch -b ece https://github.com/jinningwang/andes.git 

cd ~/repos/andes && python -m pip install --user -e .

echo "export PATH=$PATH:/home/jovyan/.local/bin" > ~/.bashrc && source ~/.bashrc

```

When done, check if your andes installation is correct by running the following command:

```bash

pip list

```

Scroll to the top and ensure that ``andes`` has an editable project location  at
`/home/jovyan/repos/andes`. You should also be able to run the `andes` command
from the command line.

## Tutorial

**Read** the example [Simulate and Plot](https://docs.andes.app/en/latest/_examples/ex1.html) to see:

- How to run a simulation
- How to plot results

### Load system

Import the `andes` package and set the verbosity level.

In [ ]:
import andes
andes.config_logger(stream_level=20)

ANDES ships with with test cases in the ``andes/cases`` folder. The cases can be
found in the [online
repository](https://github.com/cuihantao/andes/tree/master/andes/cases). One can
use `andes.get_case` to load a case from the default case folder.

ANDES configuration is loaded from the file ``$HOME/.andes/andes.rc``.
A full list of configuration options can be found in the [Config reference](https://docs.andes.app/en/latest/configref.html#config-reference) section.

If want to use default config, you can pass parameter ``default_config=True``.
For more detail, you can refer to example [Run power flow by default](https://docs.andes.app/en/latest/_examples/ex1.html#run-power-flow-by-default).

**Exercise**: Run the ANDES test case `ieee14/ieee14_ace.xlsx` from the default
case folder using **default config**.

In the following, you will replace `<**ANSWER**>` with your code.

In [ ]:
ss = andes.run(<**ANSWER**>,
               default_config=True)

The command ``andes.run()`` not only **load** system but also **solve** the power flow.
If you do not want to solve the power flow, you can use ``andes.load()`` instead.

Use ``andes.load()`` to load the system without setting up the structure of the
system. 

In [ ]:
ss = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
                default_config=True,
                setup=False)

Here you loaded a system but did not setup it, so you can add devices and setup the system manually.
More information can be found from the documentation by command ``help(andes.load)``.

In this exercise, you will implement a generator tripping by adding a `Toggle`.


In [ ]:
print(ss.Toggle.doc())

**Exercise**: Add a `Toggle` to the system. The `Toggle` is used to trip `GENROU_2` at `1.0`s. You can refeter to example [Tripping a Generator in the IEEE 14-Bus System](https://docs.andes.app/en/latest/_examples/ex9.html?highlight=ss.add).

In [ ]:
ss.add("Toggle", dict(model='SynGen', dev= < **ANSWER**>, t= < **ANSWER**>,))

You can inspect the `Toggle` by function `as_df`.

In [ ]:
ss.Toggle.as_df()

Setup the system manually.

In [ ]:
ss.setup()

Disable the first two `Toggle`s that are used for line switching.

In [ ]:
ss.Toggle.u.v[[0, 1]] = 0

### Run Analysis Routines

ANDES supports three analysis routines currently: **power flow**, **time-domain simulation (TDS)**, and **eigenvalue analysis**.

Solve power flow.

In [ ]:
ss.PFlow.run()

**Exercise**: Change the TDS finish time to `30`s. You can refer to example [Run time-domain simulation](https://docs.andes.app/en/latest/_examples/ex1.html?highlight=config.tf#id1).

In [ ]:
ss.TDS.config.tf = <**ANSWER**>

We need to disable the stability checking criteria for the TDS. The criteria stops
simulations once the maximam rotor angle deviation exceeds 180 degrees.

You can refer to [Release notes
v1.6.5](https://docs.andes.app/en/latest/release-notes.html?highlight=criteria#v1-6-5-2022-04-19).

In [ ]:
ss.TDS.config.criteria = 0

Run the TDS.

In [ ]:
ss.TDS.run()

Check if the TDS is converged. You can refer to example [Checking Exit Code](https://docs.andes.app/en/latest/getting_started/tutorial/interactive.html?highlight=exit_code#checking-exit-code).

In [ ]:
ss.exit_code

### Plot Results

ANDES provides a convenient way to plot the results. You can refer to example [Additional arguments](https://docs.andes.app/en/latest/_examples/ex1.html#additional-arguments) for more information of plot function.

Note: some Jupyter notebooks may not support inline plots. You can use the following command to enable it.

```python

import matplotlib
%matplotlib inline

```

If you want to use the more interactive backend based on Plotly, which allows
you to select and zoom, you can do the following:

```python

import matplotlib
%matplotlib widget

```

In [ ]:
import matplotlib
%matplotlib inline

**Exercise**: Plot the generator speed of ``GENROU 1``, ``GENROU 2``, and ``GENROU 3`` with scaling to 60 Hz nominal values. You can refer to examples [Plotting by variable](https://docs.andes.app/en/latest/getting_started/tutorial/interactive.html?highlight=exit_code#checking-exit-code) and [Scaling](https://docs.andes.app/en/latest/_examples/ex1.html#scaling).

In [ ]:
ss.TDS.plt.plot(ss.GENROU.omega,
                a=<**ANSWER**>,
                ycalc=<**ANSWER**>,
                )

Run eigenvalue analysis.

In [ ]:
ss.EIG.run()

**Exercise**: Plot the eigenvalue analysis results with x-axis in range ``[-1, 0.1]``. You can refer to the documentation [EIG.plot](https://docs.andes.app/en/latest/_generated/_generated/andes.routines.eig.EIG.plot.html?highlight=EIG.plot#eig-plot).

Note: one can use the Python built-in function `help` to get the documentation of a function.

```python

help(ss.EIG.plot)

```

The shorthand for this in Jupyter notebooks is `?`.

```python

?ss.EIG.plot

```


In [ ]:
ss.EIG.plot(left=<**ANSWER**>, right=<**ANSWER**>)

### Working with Data

**Exercise**: Extract the data for generator electric power ``GENROU.Pe``. You can refer to the example [Extracting Variable Data](https://docs.andes.app/en/latest/_examples/ex2.html#extracting-variable-data).

In [ ]:
Pe = ss.dae.ts.<**ANSWER**>[:, <**ANSWER**>]

Plot out the extracted data.

In [ ]:
ss.TDS.plt.plot_data(ss.dae.ts.t, Pe)

Cleanup

In [ ]:
!andes misc -C